### Read train dataset

In [1]:
import json

with open('hotpot_train_v1.1.json', 'r') as input_file:
    data = json.loads(input_file.read())

In [ ]:
data[0]

### Read preprocessed wikipedia dataset

In [ ]:
import os, bz2, json

# Walk through the extracted files
output_directory = "../data/enwiki-20171001-pages-meta-current-withlinks-processed"
def load_datas():
    datas = []
    i = 0
    for folder in os.listdir(output_directory):
        for filename in os.listdir(os.path.join(output_directory, folder)):
            if filename.endswith('.bz2'):  # Check if the file is a .bz2 file
                file_path = os.path.join(output_directory, folder, filename)
                # Decompress the .bz2 file
                try:
                    with bz2.open(file_path, 'rt') as bz2_file:  # 'rt' for text mode
                        for line in bz2_file:
                            data = json.loads(line)
                            datas.append(data)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in file {file_path}: {e}")
                except OSError as e:
                    print(f"Error opening file {file_path}: {e}")
                if i == 10:
                    return datas
                print(i)
                i += 1
    return datas
datas = load_datas()

In [ ]:
datas

In [ ]:
len(datas)

In [ ]:
datas[4566]

In [ ]:
import re

data = datas[4566]

for i, (paragraph, paragraph_offset) in enumerate(zip(data['text'], data['charoffset'])):
    sentences = []
    for sentence, sentence_list in zip(paragraph, paragraph_offset):
        sentence = re.sub(r"\s*<.*?>\s*", " ", sentence).strip()
        sentences.append(sentence)
    paragraph = ''.join(sentences)
    print(paragraph)

#### Analyze title

In [ ]:
import re
contain_title = 0
for document_id, data in enumerate(datas):
    contain_title_this_doc = False
    for i, (paragraph, paragraph_offset) in enumerate(zip(data['text'], data['charoffset'])):
        sentences = []
        for sentence, sentence_list in zip(paragraph, paragraph_offset):
            sentence = re.sub(r"\s*<.*?>\s*", " ", sentence).strip()
            sentences.append(sentence)
        paragraph = ''.join(sentences)
        if len(paragraph) < 30 and len(paragraph) != 0 and paragraph != data['title']:
            print(paragraph)
            contain_title_this_doc = True
    if contain_title_this_doc:
        print(f'document_id: {document_id}')
        contain_title += 1
print(f"{contain_title/len(datas)*100}%")

In [ ]:
for one_list in data['text']:
    print(one_list)
    for i, text in enumerate(one_list):
        print(i)
        print(text)
    print()

In [ ]:
def wikipedia_reader():
    pass

### Read raw wikipedia dataset

In [ ]:
import os, bz2, json

# Walk through the extracted files
output_directory = "../data/enwiki-20171001-pages-meta-current-withlinks-processed"
def load_datas():
    datas = []
    i = 0
    for folder in os.listdir(output_directory):
        for filename in os.listdir(os.path.join(output_directory, folder)):
            if filename.endswith('.bz2'):  # Check if the file is a .bz2 file
                file_path = os.path.join(output_directory, folder, filename)
                # Decompress the .bz2 file
                try:
                    with bz2.open(file_path, 'rt') as bz2_file:  # 'rt' for text mode
                        for line in bz2_file:
                            data = json.loads(line)
                            datas.append(data)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in file {file_path}: {e}")
                except OSError as e:
                    print(f"Error opening file {file_path}: {e}")
                if i == 10:
                    return datas
                print(i)
                i += 1
    return datas
datas = load_datas()

In [ ]:
import sys, os.path
import re
import argparse
import bz2

input_file_path = '../data/enwiki-latest-pages-articles.xml'

if input_file_path.lower().endswith(".bz2"):
    input_file = bz2.open(input_file_path, mode='rt', encoding='utf-8')
else:
    input_file = open(input_file_path)
tagRE = re.compile(r'(.*?)<(/?\w+)[^>]*>(?:([^<]*)(<.*?>)?)?')
page = []
for line in input_file:
    print(line)
    # m = tagRE.search(line)
    # if '<' not in line:         # faster than doing re.search()
    #         if page:
    #             page.append(line)
    # if m:
    #     print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    #     for v in m.groups():
    #         print(f'> {v}')
    #     print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    # print("*************************************")
    input()

In [1]:
import io
from lxml import etree

def go_over_string_element(xml_data):
    # Parse the string as a file-like object
    root = etree.fromstring(xml_data)
    redirect = root.find('redirect')
    title = root.findtext('title')
    text_element = root.find(".//text")  # Find the text element (can be nested)
    if redirect is not None or text_element is None or text_element.text is None: return None
    text = text_element.text.strip()
    print(text)
    input()
    
    return {'title': title, 'text': text}

In [ ]:
from tqdm import tqdm
from xml.etree.ElementTree import XMLPullParser
from lxml import etree

# Create an event-driven parser
raw_page = None
end_page = False
pages = []
pages_key = {}
input_file_path = '../data/enwiki-latest-pages-articles.xml'

total_iterations = 100000
parser = XMLPullParser(events=('start', 'end'))

with tqdm(total=total_iterations, desc="Processing") as pbar:
    with open(input_file_path, 'r') as input_file:
        for line in input_file:
            parser.feed(line)
            # print(line.strip())
            for event, element in parser.read_events():
                key = element.tag.split('}')[-1]
                if event == 'start' and 'page' in element.tag:
                    raw_page = []
                if event == 'end' and 'page' in element.tag:
                    end_page = True
            # add line
            if isinstance(raw_page, list):
                raw_page.append(line)

            # parse string when end page
            if end_page:
                raw_page = ''.join(raw_page)
                page = go_over_string_element(raw_page)
                if page:
                    for key in page:
                        pages_key[key] = pages_key.get(key, 0) + 1
                    pages.append(page)
                    pbar.set_postfix(pages_key)
                    pbar.update(1)
                    # input()
                if len(pages) == total_iterations:
                    break
                raw_page = None
                end_page = False
                    
        

In [ ]:
pages_key

In [17]:
for page in pages:
    if page == {}:
        print(page)

In [8]:
for page in pages[:1]:
    sections = {}
    page_text = page['text']
    abstract_pattern = r"\n'''(.*?)'''(.*?)(?=\n\=|$)"
    abstract_match = re.search(abstract_pattern, pages[0]['text'], re.DOTALL)
    abstract = abstract_match.group(0).strip() if abstract_match else None
    sections['abstract'] = abstract
    
    raw_sections = re.split(r'(?m)^==\s*', pages[10]['text'])
    old_section_title = None
    for section in raw_sections[1:]:
        lines = section.split('\n')
        section_title = lines[0].strip('= ')
        section_content = "\n".join(lines[1:])
        if not section.startswith('='):
            sections[section_title] = section_content
            old_section_title = section_title
        else:
            assert old_section_title is not None, 'old_section_title shouldn\'t be None'
            sections[old_section_title] += f'\nsub title: {section_title}\n{section_content}'

In [ ]:
for k, v in sections.items():
    print(k, v)

In [ ]:
pages[0]

In [ ]:
import re

def _clean_text(raw_text):
    print(raw_text)
    cleaned_text = re.sub(r'\{\{.*?\}\}', '', raw_text, flags=re.DOTALL)
    print(cleaned_text)
    cleaned_text = re.sub(r"'''(.*?)'''", r'\1', cleaned_text)
    print(cleaned_text)
    cleaned_text = re.sub(r'\[\[(?:[^\]|]+\|)?([^\]]+)\]\]', r'\1', cleaned_text)
    print(cleaned_text)
    input()
    return cleaned_text

def _read_page(page):
    sections = {}
    page_text = page['text']
    abstract_pattern = r"\n'''(.*?)'''(.*?)(?=\n\=|$)"
    abstract_match = re.search(abstract_pattern, page_text, re.DOTALL)
    abstract = abstract_match.group(0).strip() if abstract_match else None
    sections['abstract'] = _clean_text(abstract)
    
    raw_sections = re.split(r'(?m)^==\s*', page_text)
    old_section_title = None
    for section in raw_sections[1:]:
        lines = section.split('\n')
        section_title = lines[0].strip('= ')
        section_content = _clean_text("\n".join(lines[1:]))
        if not section.startswith('='):
            sections[section_title] = section_content
            old_section_title = section_title
        else:
            assert old_section_title is not None, 'old_section_title shouldn\'t be None'
            sections[old_section_title] += f'\nsub title: {section_title}\n{section_content}'
    

In [ ]:
for k, v in sections.items():
    print(k, v)

In [ ]:
import re

short_descriptions = []
abstracts = []
sections = []
sections_of_one_document = []

for text in pages[1]['text'].split('\n'):
    if 'short description' in text.lower():
        result = re.search(r'{{\s*short description\s*\|([^}]+)}}', text, re.IGNORECASE)
        if result:
            short_description = result.group(1).strip()  # Extract the description
            short_descriptions.append(short_description)
            print(short_description)

In [6]:
for page in pages:
    if 'redirect' in page:
        for k, v in page.items():
            if isinstance(v, str):
                print(v)
            else:
                print(f'{k}, text: {v.text}')

### Check the distribution of the wikipedia dump

In [2]:
import os
import json
from tqdm import tqdm
from string_preprocessing import _go_over_string_element, remove_self_closing_ref, remove_ref_with_content, remove_braces_content, remove_square_brackets, _clean_text, _get_abstract, _read_page

titles = {}
length = []
section_titles = {}

data_dir = '/home/zhengzheng/scratch0/projects/Fine-Tuned-GPT-2-with-articles-ground-truth-main/code/llamaIndex/.cache'
filenames = sorted(list(os.listdir(data_dir)), key=lambda x: x.split('_')[-1].split('.')[0])
save_file = open('temp.txt', 'w')
for filename in filenames[:1]:
    file_path = os.path.join(data_dir, filename)
    file_size = os.path.getsize(file_path)
    with open(file_path, 'r') as input_file:
        with tqdm(total=file_size, desc=f'Loading {file_path.split(os.path.sep)[-1]}', unit='B', unit_scale=True, unit_divisor=1024) as pbar:
            for i, line in enumerate(input_file):
                if i < 71:
                    continue
                pbar.update(len(line))
                data = json.loads(line)
                page = _go_over_string_element(data['page'])
                if page:
                    save_file.write(page['text'])
                    save_file.flush()
                    break
                    paper_content, file_dict, print_str = _read_page(page)
                    if paper_content:
                        save_file.write(page['text'] + '\n=====================================\n' + paper_content)
                        save_file.flush()
                        # titles[file_dict['title']] = titles.get(file_dict['title'], 0) + 1
                        # length.append(len(paper_content))
                        # for key in file_dict['sections']:
                        #     section_titles[key] = section_titles.get(key, 0) + 1
            

Loading raw_page_0.jsonl:   0%|          | 29.4k/6.99G [00:00<15:20, 8.16MB/s]


In [1]:
import html
from string_preprocessing import _go_over_string_element, remove_self_closing_ref, remove_ref_with_content, remove_braces_content, remove_square_brackets, _clean_text, _get_abstract, _read_page

text = r"""An important relationship between an object's astronomical (geometric) albedo, [[Absolute magnitude#Absolute magnitude for planets (H)|absolute magnitude]] and diameter is given by:<ref name="bruton">{{cite web
 |title=Conversion of Absolute Magnitude to Diameter for Minor Planets
 |publisher=Department of Physics & Astronomy (Stephen F. Austin State University)
 |author=Dan Bruton
 |url=http://www.physics.sfasu.edu/astro/asteroids/sizemagnitude.html
 |access-date=7 October 2008
 |archive-url=https://web.archive.org/web/20081210190134/http://www.physics.sfasu.edu/astro/asteroids/sizemagnitude.html
 |archive-date=10 December 2008
 |url-status=dead
}}</ref>
<math display="block">A =\left ( \frac{1329\times10^{-H/5}}{D} \right ) ^2,</math>
where <math>A</math> is the astronomical albedo, <math>D</math> is the diameter in kilometers, and <math>H</math> is the absolute magnitude.
"""

# cleaned_text = html.unescape(text)
# # # Remove <ref> tag without greedy
# cleaned_text = remove_self_closing_ref(cleaned_text)
# # print(cleaned_text)
# cleaned_text = remove_ref_with_content(cleaned_text)
# print(cleaned_text)

# stack = []
# result = []
# i = 0
# text = cleaned_text

# while i < len(text):
#     char = text[i]
#     if char == '{' and i + 1 < len(text) and text[i + 1] == '{':
#         # Push the current length of result onto the stack to remember where the brace started
#         stack.append(len(result))
#         i += 1  # Skip the second '{'
#     elif char == '}' and i + 1 < len(text) and text[i + 1] == '}' and len(stack) > 0:
#         print(''.join(result))
#         print()
#         print(len(stack))
#         print("\n==========================================\n\n")
        
#         start = stack.pop()
#         block_content = ''.join(result[start:])
#         if '|' in block_content and block_content[0] == 'blackquate' and len(block_content.split('|')) == 2:
#             keep_content = block_content.split('|')[1]
#             result = result[:start] + list(keep_content)
#         else:
#             result = result[:start]
#         i += 1  # Skip the second '}'
#     else:
#         # Append character to the result only if not within braces
#         result.append(char)
#     i += 1
# print(''.join(result))

# stack = []
# result = []
# for char in cleaned_text:
#     if char == '{':
#         # Push the current length of result onto the stack to remember where the brace started
#         stack.append(len(result))
#     elif char == '}':
#         print("get }")
#         print(''.join(result))
#         start = stack.pop()
#     else:
#         # Append character to the result only if not within braces
#         if len(stack) == 0:
#             result.append(char)

_clean_text(text)

'An important relationship between an object\'s astronomical (geometric) albedo, absolute magnitude and diameter is given by:\n<math display="block">A =\\left ( \\frac{1329\\times10^{-H/5}}{D} \\right ) ^2,</math>\nwhere <math>A</math> is the astronomical albedo, <math>D</math> is the diameter in kilometers, and <math>H</math> is the absolute magnitude.\n'

In [3]:
!pwd


/scratch0/zhengzheng/projects/read_wikipedia/code


In [2]:
import json

with open('./processed_data.json', 'r') as input_file:
    output = json.loads(input_file.read())
    titles = output['titles']
    length = output['length']
    section_titles = output['section_titles']

In [ ]:
num = 0
for title in titles:
    if title.lower().startswith('file:'):
        print(title)
        num += 1

In [ ]:
num

In [ ]:
for title in sorted(list(section_titles.items()), key=lambda x: x[1], reverse=True)[:30]:
    print(title)

In [ ]:
sorted(length, reverse=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a violin plot
sns.violinplot(y=length, color='skyblue', inner='box')

# Add labels and title
plt.xlabel('Frequency')
plt.ylabel('Numbers')
plt.title('Violin Plot of Number Distribution')

# Show the plot
plt.show()

In [ ]:
# Your dictionary
my_dict = {
    'references': 241830,
    'External links': 182700,
    'See also': 138906,
    'Further reading': 40252,
    'Introduction': 12345,
}

# Keys to remove
keys_to_remove = ['References', 'External links', 'See also', 'Further reading']

# Remove keys safely
for key in keys_to_remove:
    my_dict.pop(key, None)  # None is the default value to avoid errors if the key doesn't exist

# Output the modified dictionary
print(my_dict)
